# pyhf example

This notebook runs the examples provided inside: 
[https://pyhf.github.io/pyhf-tutorial/introduction.html](https://pyhf.github.io/pyhf-tutorial/introduction.html)

Useful documentation can also be found here: 
[https://pyhf.readthedocs.io/_/downloads/en/latest/pdf/](https://pyhf.readthedocs.io/_/downloads/en/latest/pdf/)

In [4]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil


Let us build out Poisson model first with a two-bin channel ('singlechannel') and containing two samples ('signal' and 'background'):

In [46]:
model=pyhf.simplemodels.uncorrelated_background(
    signal=[5.0, 10.0], bkg=[50.0, 60.0], bkg_uncertainty=[0.0, 0.0])
print(model)
print(json.dumps(model.spec, indent=2))
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"  nauxdata: {model.config.nauxdata}")
print(f"   auxdata: {model.config.auxdata}")

{
  "channels": [
    {
      "name": "singlechannel",
      "samples": [
        {
          "name": "signal",
          "data": [
            5.0,
            10.0
          ],
          "modifiers": [
            {
              "name": "mu",
              "type": "normfactor",
              "data": null
            }
          ]
        },
        {
          "name": "background",
          "data": [
            50.0,
            60.0
          ],
          "modifiers": [
            {
              "name": "uncorr_bkguncrt",
              "type": "shapesys",
              "data": [
                0.0,
                0.0
              ]
            }
          ]
        }
      ]
    }
  ]
}
  channels: ['singlechannel']
     nbins: {'singlechannel': 2}
   samples: ['background', 'signal']
 modifiers: [('mu', 'normfactor'), ('uncorr_bkguncrt', 'shapesys')]
parameters: ['mu', 'uncorr_bkguncrt']
  nauxdata: 0
   auxdata: []


In [47]:
model.expected_data([1.0, 1.0, 1.0], include_auxdata=True)

array([55., 70.])

In [48]:
print(model.config.parameters)
print(model.config.modifiers)
print(model.config.par_order)


['mu', 'uncorr_bkguncrt']
[('mu', 'normfactor'), ('uncorr_bkguncrt', 'shapesys')]
['mu', 'uncorr_bkguncrt']


In [49]:
init_pars = model.config.suggested_init()
print(init_pars)
model.expected_actualdata(init_pars)


[1.0]


array([55., 70.])

In [19]:
model.config.suggested_bounds()


[(0, 10), (1e-10, 10.0), (1e-10, 10.0)]

In [50]:
bkg_pars = init_pars.copy()
bkg_pars[model.config.poi_index] = 0
model.expected_actualdata(bkg_pars)


array([50., 60.])

In [51]:
observations = [53.0, 65.0] + model.config.auxdata
model.logpdf(pars=bkg_pars, data=observations)


array([-6.20409914])

In [52]:
pyhf.infer.mle.fit(data=observations, pdf=model)


array([0.52366294])

In [53]:
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    [53.0, 65.0] + model.config.auxdata,
    model,
    test_stat="qtilde",
    return_expected_set=True,
    calctype="toybased",
    ntoys=1000,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")


      Observed CLs: 0.3222
Expected CLs(-2 σ): 0.0476
Expected CLs(-1 σ): 0.0777
Expected CLs( 0 σ): 0.1819
Expected CLs( 1 σ): 0.3550
Expected CLs( 2 σ): 1.0000
